In [ ]:
import torch

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.autograd import Variable

def cbr_layer(in_channels, out_channels, kernel_size, groups=1, stride=1, activation=True):
    if activation:
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=int(kernel_size / 2),
                      groups=groups, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6(inplace=True))
    else:
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=int(kernel_size / 2),
                      groups=groups,
                      bias=False),
            nn.BatchNorm2d(out_channels, affine=True, eps=1e-5, momentum=0.1))


class InvertedResidualBlock(nn.Module):
    def __init__(self, in_planes, out_planes, exp, stride):
        super().__init__()
        intermed_planes = in_planes*exp
        self.residual = (in_planes == out_planes) and (stride == 1)
        self.output = nn.Sequential(cbr_layer(in_planes, intermed_planes, kernel_size=1),
                                     cbr_layer(intermed_planes, intermed_planes, kernel_size=3, stride=stride, groups=intermed_planes),
                                     cbr_layer(intermed_planes, out_planes, kernel_size=1, activation=False))

    def forward(self, x):
        out = self.output(x)
        if self.residual:
            return (out+x)
        else:
            return out

class Mobilenet_backbone(nn.Module):
    def __init__(self):
        super().__init__()
        mobilenet_config = [[1, 16, 1, 1],
                            [6, 24, 2, 2],
                            [6, 32, 3, 2],
                            [6, 64, 4, 2],
                            [6, 96, 3, 1],
                            [6, 160, 3, 2],
                            [6, 320, 1, 1]]

        self.in_channels = 32
        self.layer1 = cbr_layer(3, self.in_channels, kernel_size=3, stride=2)
        layer_count = 2
        for t, c, n, s in mobilenet_config:
            layers = []
            for i in range(n):
                layers.append(InvertedResidualBlock(self.in_channels, c, t, stride=s if i==0 else 1))
                self.in_channels = c
            setattr(self, "layer{}".format(layer_count), nn.Sequential(*layers))
            layer_count += 1

    def forward(self,x):
        l1 = self.layer1(x)
        l2 = self.layer2(l1)
        l3 = self.layer3(l2)  # 24, x / 4
        l4 = self.layer4(l3)  # 32, x / 8
        l5 = self.layer5(l4)  # 64, x / 16
        l6 = self.layer6(l5)  # 96, x / 16
        l7 = self.layer7(l6)  # 160, x / 32
        l8 = self.layer8(l7)  # 320, x / 32

        return l3, l4, l5, l6, l7, l8
class CRPBlock(nn.Module):
    def __init__(self, in_planes, out_planes, num_stages, groups=False):
        super().__init__()
        for i in range(num_stages):
            setattr(self, '{}_{}'.format(i+1, 'outvar_dimred'),
                    nn.Conv2d(in_planes if (i == 0) else out_planes,
                              out_planes, kernel_size=1, stride=1,
                              padding=0, bias=False, groups=in_planes if groups else 1))
        self.stride = 1
        self.n_stages = num_stages
        self.maxpool = nn.MaxPool2d(kernel_size=5, stride=1, padding=2)

    def forward(self,x):
        out=x
        for i in range(self.n_stages):
            out = self.maxpool(out)
            out = getattr(self, '{}_{}'.format(i+1, 'outvar_dimred'))(out)
            x = out+x
        return x

class RefineNetDecoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.conv8 = nn.Conv2d(320, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv7 = nn.Conv2d(160, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv6 = nn.Conv2d(96, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv5 = nn.Conv2d(64, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv4 = nn.Conv2d(32, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv3 = nn.Conv2d(24, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.crp4 = self.make_crp(256, 256, 4, groups=False)
        self.crp3 = self.make_crp(256, 256, 4, groups=False)
        self.crp2 = self.make_crp(256, 256, 4, groups=False)
        self.crp1 = self.make_crp(256, 256, 4, groups=True)
        self.conv_adapt4 = nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv_adapt3 = nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.conv_adapt2 = nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=0, groups=1, bias=False)
        self.relu = nn.ReLU6(inplace=True)
        self.pre_depth = nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=0, groups=256, bias=False)
        self.depth = nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)
        self.pre_segm = nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=0, groups=256, bias=False)
        self.segm = nn.Conv2d(256, self.num_classes, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)

    def make_crp(self, in_planes, out_planes, num_stages, groups=False):
        layers = [CRPBlock(in_planes, out_planes, num_stages, groups=groups)]
        return nn.Sequential(*layers)

    def forward(self, l3, l4, l5, l6, l7, l8):
        l8 = self.conv8(l8)
        l7 = self.conv7(l7)
        l7 = self.relu(l8+l7)
        l7 = self.crp4(l7)
        l7 = self.conv_adapt4(l7)
        l7 = nn.Upsample(size = l6.size()[2:],mode='bilinear', align_corners=False)(l7)

        l6 = self.conv6(l6)
        l5 = self.conv5(l5)
        l5 = self.relu(l5+l6+l7)
        l5 = self.crp3(l5)
        l5 = self.conv_adapt3(l5)
        l5 = nn.Upsample(size = l4.size()[2:],mode='bilinear', align_corners=False)(l5)
        l4 = self.conv4(l4)
        l4 = self.relu(l5+l4)
        l4 = self.crp2(l4)
        l4 = self.conv_adapt2(l4)
        l4 = nn.Upsample(size = l3.size()[2:],mode='bilinear', align_corners=False)(l4)

        l3 = self.conv3(l3)
        l3 = self.relu(l3+l4)
        l3 = self.crp1(l3)

        out_segm = self.pre_segm(l3)
        out_segm = self.relu(out_segm)
        out_segm = self.segm(out_segm)

        out_depth = self.pre_depth(l3)
        out_depth = self.relu(out_depth)
        out_depth = self.depth(out_depth)

        return out_depth, out_segm


class Hydranet(nn.Module):
    def __init__(self, num_tasks, num_classes):
        super().__init__()
        self.n_classes = num_classes
        self.n_tasks = num_tasks
        self.enc = Mobilenet_backbone()
        self.dec = RefineNetDecoder(self.n_classes)

    def forward(self, x):
        l3, l4, l5, l6, l7, l8 = self.enc(x)
        out_depth, out_segm = self.dec(l3, l4, l5, l6, l7, l8)
        return out_depth, out_segm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import glob
import numpy as np
import cv2
import matplotlib.cm as cm
import matplotlib.colors as co

import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.autograd import Variable
#from Hydranet import Hydranet


hydranet = Hydranet(2,6)
ckpt = torch.load("/content/ExpKITTI_joint.ckpt", map_location='cpu')
hydranet.enc.load_state_dict(ckpt["state_dict"], strict=False)
hydranet.dec.load_state_dict(ckpt["state_dict"], strict=False)
hydranet.eval()

IMG_SCALE  = 1./255
IMG_MEAN = np.array([0.485, 0.456, 0.406]).reshape((1, 1, 3))
IMG_STD = np.array([0.229, 0.224, 0.225]).reshape((1, 1, 3))
CMAP = np.load('/content/cmap_nyud.npy')
NUM_CLASSES = 6


def prepare_img(img):
    return (img * IMG_SCALE - IMG_MEAN) / IMG_STD


def pipeline(img):
    with torch.no_grad():
        img_var = Variable(torch.from_numpy(prepare_img(img).transpose(2,0,1)[None]), requires_grad=False).float()
        if torch.cuda.is_available():
            img_var = img_var.cuda()
        depth, segm = hydranet(img_var)
        segm = cv2.resize(segm[0].cpu().data.numpy().transpose(1,2,0), img.shape[:2][::-1], interpolation=cv2.INTER_CUBIC)
        depth = cv2.resize(depth[0].cpu().data.numpy().transpose(1,2,0), img.shape[:2][::-1], interpolation=cv2.INTER_CUBIC)
        segm = CMAP[segm.argmax(axis=2)].astype(np.uint8)
        depth = np.abs(depth).astype(np.uint8)
        return depth, segm

def project_to_image(img, depth, seg):
    cmap_proj = plt.cm.get_cmap("hsv", 256)
    cmap_proj = np.array([cmap_proj(i) for i in range(256)])[:, :3] * 255
    img_copy = cv2.addWeighted(img, 0.8, seg, 1.5, 0.7)
    for i in range(depth.shape[0]):
        for j in range(depth.shape[1]):
            num = np.random.randint(1, 20)
            num1 = np.random.randint(1, 20)
            if i>100 and (j%num==0 or i%num1==0) and depth[i][j]>5:
                if num<8:
                    continue
                point_depth = depth[i][j]
                color = cmap_proj[int(510/point_depth),:]
                cv2.circle(img_copy, (j,i), 2, tuple(color), thickness=-1)
    return img_copy

def depth_to_rgb(depth):
    normalizer = co.Normalize(vmin=0, vmax=80)
    mapper = cm.ScalarMappable(norm=normalizer, cmap='plasma')
    colormapped_im = (mapper.to_rgba(depth)[:, :, :3] * 255).astype(np.uint8)
    return colormapped_im

video_files = sorted(glob.glob("demo/*.png"))
result_video = []
projected_video = []
count=0
for idx, img_path in enumerate(video_files):
    image = np.array(Image.open(img_path))
    h, w, _ = image.shape
    depth, seg = pipeline(image)
    projected_output = project_to_image(image, depth, seg)
    result_video.append(cv2.cvtColor(cv2.vconcat([image, seg, depth_to_rgb(depth)]), cv2.COLOR_BGR2RGB))
    projected_video.append(cv2.cvtColor(projected_output, cv2.COLOR_BGR2RGB))
    count+=1

out = cv2.VideoWriter('/content/out.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, (w,3*h))
proj = cv2.VideoWriter('/content/proj.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 15, (w,h))

for i in range(len(result_video)):
    out.write(result_video[i])
out.release()


for i in range(len(projected_video)):
    proj.write(projected_video[i])
proj.release()